#### Requirements

In [1]:
import sys
import os

import geopandas as gpd
import pandas as pd
import numpy as np
import datetime as dt

from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

#### Constants

In [58]:
# General
BASE_PATH = os.path.dirname(os.getcwd())

# Mapping
CHANGE_TYPE_MAP = {'Demolition': 0, 'Road': 1, 'Residential': 2, 'Commercial': 3, 'Industrial': 4,
       'Mega Projects': 5}

# Data
COLORS = ['red', 'green', 'blue']
METRICS = ['std', 'mean']

# Columns groups
COLUMNS_TO_DROP = ['index']
DATE_COLUMNS = ['date0', 'date1', 'date2', 'date3', 'date4']

#### Data preprocessing

In [3]:
## Read data
train_df = gpd.read_file(f'{BASE_PATH}/data/train.geojson', index_col=0)
# test_df = gpd.read_file(f'{BASE_PATH}/data/test.geojson', index_col=0)

In [16]:
#train_df['geography_type'].str.split(',').explode().str.strip().unique()
#geography_types = ['Dense Forest', 'Grass Land', 'Sparse Forest', 'Farms', 'River',
#       'Coastal', 'Lakes', 'Barren Land', 'N,A', 'Desert', 'Hills',
#       'Snow'] 
################## Pensar no que fazer com esses caras de tipo 'N,A' #########################
train_df[train_df['geography_type'].str.contains('N,A')]['geography_type'].unique()

array(['Dense Forest', 'Grass Land', 'Sparse Forest', 'Farms', 'River',
       'Coastal', 'Lakes', 'Barren Land', 'N', 'A', 'Desert', 'Hills',
       'Snow'], dtype=object)

In [60]:
train_df[DATE_COLUMNS] = train_df[DATE_COLUMNS].apply(pd.to_datetime)

C:\Users\Joao Pedro\AppData\Local\Temp\ipykernel_15708\1377944601.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  train_df[DATE_COLUMNS] = train_df[DATE_COLUMNS].apply(pd.to_datetime)
C:\Users\Joao Pedro\AppData\Local\Temp\ipykernel_15708\1377944601.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  train_df[DATE_COLUMNS] = train_df[DATE_COLUMNS].apply(pd.to_datetime)
C:\Users\Joao Pedro\AppData\Local\Temp\ipykernel_15708\1377944601.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  train_df[DATE_COLUMNS] = train_df[DATE_COLUMNS].apply(pd.to_datetime

In [11]:
# Create new polygon features
train_df['area'] = train_df['geometry'].area
train_df['length'] = train_df['geometry'].length
train_df['centroid'] = train_df['geometry'].centroid

# Create new date related features
for metric in METRICS:
    for color in COLORS:
        for i in range(2, 6):
            delta = train_df[f'img_{color}_{metric}_date{i}'] - train_df[f'img_{color}_{metric}_date{i-1}']
            train_df[f'img_{color}_{metric}_delta{i}'] = delta
for i in range(1, 5):
    train_df[f'date_delta{i}'] = train_df[f'date{i}'] - train_df[f'date{i-1}']

In [38]:
# Map features
train_df['change_type'].map(CHANGE_TYPE_MAP)

In [ ]:
# Drop useless features
train_df

In [ ]:
# Write in excel
train_df.to_excel('train_head.xlsx', index=False)
test_df.to_excel('test_head.xlsx', index=False)